Import

In [ ]:
import os
from contextlib import redirect_stdout

import sys
sys.path.append('./coeqwalpackage')

import numpy as np
import pandas as pd
import datetime as dt
import re
from coeqwalpackage.metrics import *
from coeqwalpackage.tier import calc_indelta_tier, generate_salinity_tier_assignment_matrix
import cqwlutils as cu
import plotting as pu
from collections import OrderedDict

### Initialize

In [ ]:
CtrlFile = 'CalSim3DataExtractionInitFile_v4.xlsx'
CtrlTab = 'Init'

ScenarioListFile, ScenarioListTab, ScenarioListPath, DVDssNamesOutPath, SVDssNamesOutPath, ScenarioIndicesOutPath, DssDirsOutPath, VarListPath, VarListFile, VarListTab, VarOutPath, DataOutPath, ConvertDataOutPath, ExtractionSubPath, DemandDeliverySubPath, ModelSubPath, GroupDataDirPath, ScenarioDir, DVDssMin, DVDssMax, SVDssMin, SVDssMax, NameMin, NameMax, DirMin, DirMax, IndexMin, IndexMax, StartMin, StartMax, EndMin, EndMax, VarMin, VarMax, DemandFilePath, DemandFileName, DemandFileTab, DemMin, DemMax, InflowOutSubPath, InflowFilePath, InflowFileName, InflowFileTab, InflowMin, InflowMax = cu.read_init_file(CtrlFile, CtrlTab)

### Read scenario indices

In [ ]:
indexhdr, index_name = cu.read_from_excel(ScenarioListPath, ScenarioListTab, IndexMin, IndexMax, hdr=True)
index_names = []
for i in range(len(index_name)):
    index_names.append(index_name[i][0])
index_names

### Read dataset

In [ ]:
df, dss_names = read_in_df(ConvertDataOutPath,DVDssNamesOutPath)
df = add_water_year_column(df)
df

### Define variables

In [ ]:
in_delta_vars = ["EM_EC_MONTH", "JP_EC_MONTH"]
export_vars = ["TRACYEC_MAX14DAY", "BANKSEC_MAX14DAY"]
indelta_thresholds={"Top": 2500, "Mid": 1600, "Low": 900}
export_thresholds={"Top": 2500, "Mid": 1600, "Low": 900}
indelta_station_list=["EM", "JP"]
export_station_list=["BANKSEC", "TRACYEC"]
indelta_rules = OrderedDict([
    (1, {"LT_A": 0.75, "LT_B": None, "GT_C": 0.05}),
    (2, {"LT_A": 0.65, "LT_B": 0.75, "GT_C": 0.12}),
    (3, {"LT_A": 0.55, "LT_B": 0.65, "GT_C": 0.20}),
])
x2 = 'X2_PRV_KM'
compliance_points_indelta = ["EM_EC_MONTH", "JP_EC_MONTH", "RS_EC_MONTH", "CO_EC_MONTH"]
compliance_points_export = ["BANKSEC", "TRACYEC"]

### Subset data set

In [ ]:
in_delta_df = create_subset_list(df, in_delta_vars)
in_delta_df

In [ ]:
export_df = create_subset_list(df, export_vars)
export_df

### X2

In [ ]:
# Annual Average
april_x2_ann_avg = compute_annual_means(df, x2, units="KM", months=[4])
september_x2_ann_avg = compute_annual_means(df, x2, units="KM", months=[9])

# Annual CV
april_x2_ann_cv = compute_cv(df, x2, "April_X2_CV", months=[4], units="KM")
april_x2_ann_cv.index.name = 'Scenario'
september_x2_ann_cv = compute_cv(df, x2, "September_X2_CV", months=[9], units="KM")
september_x2_ann_cv.index.name = 'Scenario'

In [ ]:
# Clean up dataframes for export
april_x2_ann_avg.columns = 'AprilX2_' + april_x2_ann_avg.columns.get_level_values('B').str.split('_').str[3]
april_x2_ann_avg.columns.name = None

september_x2_ann_avg.columns = 'SeptemberX2_' + september_x2_ann_avg.columns.get_level_values('B').str.split('_').str[3]
september_x2_ann_avg.columns.name = None

### Salinity at compliance points

In [ ]:
salinity_compliance_points_indelta_df = create_subset_list(df, compliance_points_indelta)
salinity_compliance_points_export_df = create_subset_list(df, compliance_points_export)

In [ ]:
# Clean up dataframes for export
salinity_compliance_points_indelta_df.columns = salinity_compliance_points_indelta_df.columns.get_level_values('B')
salinity_compliance_points_indelta_df.columns.name = None

salinity_compliance_points_export_df.columns = salinity_compliance_points_export_df.columns.get_level_values('B')
salinity_compliance_points_export_df.columns.name = None

### Tier calculation functions

In [ ]:
# calc_indelta_tier() is now imported from coeqwalpackage.tier

In [ ]:
# generate_salinity_tier_assignment_matrix() is now imported from coeqwalpackage.tier

### In-Delta tier assignment (new version):

In [ ]:
tiers = {} #<- dictionary to store results
for scenID in index_names: # iterate through the list of scenario IDs
    # call the function defined above
    tiers[scenID] = calc_indelta_tier(df = in_delta_df, scenID = scenID, stations = in_delta_vars, thresholds =  indelta_thresholds, tier_rules= indelta_rules)
    # print out the progress
    print(f"assigned tier {tiers[scenID]} to scenario {scenID}")

# create a dataframe from the dictionary - index is the scenario id, 
# single column is the tier value
tier_indelta_df = pd.DataFrame.from_dict(tiers, orient='index', columns=['Salinity_Tier']) #, index='ScenarioID')
tier_indelta_df.index.name = 'ScenarioID' #rename the column

In [ ]:
tier_indelta_df

### In-Delta tier assignment (old version):

In [ ]:
# tier_indeltaold_df = generate_salinity_tier_assignment_matrix(
#     df=in_delta_df,
#     station_list=indelta_station_list,
#     thresholds=indelta_thresholds,
#     start_date="1921-10-01"
# )

In [ ]:
# tier_indeltaold_df

### Export tier assignment:

In [ ]:
tier_export_df = generate_salinity_tier_assignment_matrix(
    df=export_df,
    station_list=export_station_list,
    thresholds=export_thresholds,
    start_date="1921-10-01"
)

In [ ]:
tier_export_df

### Save Files

In [ ]:
# Use find_calsim_model_root from cqwlutils
calsim_base_path = cu.find_calsim_model_root()

salinity_output_dir = os.path.join(
    calsim_base_path,
    "Scenarios",
    "Performance_Metrics",
    "Tiered_Outcome_Measures",
    "Salinity"
)
os.makedirs(salinity_output_dir, exist_ok=True)

april_x2_ann_avg_path = os.path.join(salinity_output_dir, "AprilX2_AnnualAverage.csv")
april_x2_ann_avg.to_csv(april_x2_ann_avg_path, index=True)

april_x2_ann_cv_path = os.path.join(salinity_output_dir, "AprilX2_AnnualCV.csv")
april_x2_ann_cv.to_csv(april_x2_ann_cv_path, index=True)

september_x2_ann_avg_path = os.path.join(salinity_output_dir, "SeptemberX2_AnnualAverage.csv")
september_x2_ann_avg.to_csv(september_x2_ann_avg_path, index=True)

september_x2_ann_cv_path = os.path.join(salinity_output_dir, "SeptemberX2_AnnualCV.csv")
september_x2_ann_cv.to_csv(september_x2_ann_cv_path, index=True)

salinity_compliance_points_indelta_path = os.path.join(salinity_output_dir, "InDeltaSalinity.csv")
salinity_compliance_points_indelta_df.to_csv(salinity_compliance_points_indelta_path, index=True)

salinity_compliance_points_export_path = os.path.join(salinity_output_dir, "ExportSalinity.csv")
salinity_compliance_points_export_df.to_csv(salinity_compliance_points_export_path, index=True)

In [ ]:
salinity_output_path = os.path.join(salinity_output_dir, "InDeltaTierAssignment.csv")
tier_indelta_df.to_csv(salinity_output_path, index=True)

salinity_output_path = os.path.join(salinity_output_dir, "ExportTierAssignment.csv")
tier_export_df.to_csv(salinity_output_path, index=True)

In [ ]:
print(salinity_output_dir)

In [ ]:
print("Done!")